Import required libraries

In [1]:
import numpy as np
import cv2

Define global variables

In [2]:
MIN_CONTOUR_AREA = 80                       # minimum area of valid contour
MAX_CONTOUR_AREA = 1200                     # maximum area of valid contour

RESIZED_IMAGE_WIDTH = 20                    # width of resized image
RESIZED_IMAGE_HEIGHT = 30                   # height of resized image

TRAINING_IMAGES_PATH = 'train_images'                                       # folder with training images
TESTING_IMAGES_PATH = 'test_images'                                         # folder with testing images
TRAINING_ALPHA = TRAINING_IMAGES_PATH + '/' + "train_alpha2.png"            # training alphabet image
TRAINING_DIGIT = TRAINING_IMAGES_PATH + '/' + "train_digit2.png"            # training digit image
TEST_IMAGE = TESTING_IMAGES_PATH + '/' + "test1.png"                        # test image

Define contour class to store characters in test image

In [3]:
class ContourWithData():
    npaContour = None           # contour
    boundingRect = None         # bounding rect for contour
    intRectX = 0                # bounding rect top left corner x location
    intRectY = 0                # bounding rect top left corner y location
    intRectWidth = 0            # bounding rect width
    intRectHeight = 0           # bounding rect height
    fltArea = 0.0               # area of contour

    # calculate bounding rect info
    def calculateRectTopLeftPointAndWidthAndHeight(self):
        [intX, intY, intWidth, intHeight] = self.boundingRect
        self.intRectX = intX
        self.intRectY = intY
        self.intRectWidth = intWidth
        self.intRectHeight = intHeight

    # this is oversimplified, for a production grade program
    def checkIfContourIsValid(self):
        if self.fltArea < MIN_CONTOUR_AREA or self.fltArea > MAX_CONTOUR_AREA: 
            return False
        return True

### TRAIN MODEL

Open training characters image

In [ ]:
imgTrainingAlpha = cv2.imread(TRAINING_ALPHA)               # read in training alpha image
imgTrainingDigit = cv2.imread(TRAINING_DIGIT)               # read in training digit image

if imgTrainingAlpha is None:                                # if image was not read successfully
    # print error message to std out
    print("error: alpha image not read from file \n\n")
    # and exit function (which exits program)
    exit(0)
# end if
if imgTrainingDigit is None:                                # if image was not read successfully
    # print error message to std out
    print("error: digit image not read from file \n\n")
    # and exit function (which exits program)
    exit(0)
# end if

Modify training alphabet image

In [ ]:
imgGrayAlpha = cv2.cvtColor(imgTrainingAlpha, cv2.COLOR_BGR2GRAY)               # get grayscale image
imgBlurredAlpha = cv2.GaussianBlur(imgGrayAlpha, (5, 5), 0)                     # blur

# filter image from grayscale to black and white
imgThreshAlpha = cv2.adaptiveThreshold( imgBlurredAlpha,
                                        # make pixels that pass the threshold full white
                                        255,                                  
                                        # use gaussian rather than mean, seems to give better results
                                        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        # invert so foreground will be white, background will be black
                                        cv2.THRESH_BINARY_INV,
                                        # size of a pixel neighborhood used to calculate threshold value
                                        11,
                                        # constant subtracted from the mean or weighted mean
                                        2)                                    

Find seperate characters from alphabet image

In [ ]:
npaContoursAlpha, _ = cv2.findContours(# input image, use a copy since the function will modify this image in the course of finding contours
                                       imgThreshAlpha.copy(),
                                       # retrieve the outermost contours only
                                       cv2.RETR_EXTERNAL,
                                       # compress horizontal, vertical, and diagonal segments and leave only their end points
                                       cv2.CHAIN_APPROX_SIMPLE)

Modify training digit image

In [ ]:
imgGrayDigit = cv2.cvtColor(imgTrainingDigit, cv2.COLOR_BGR2GRAY)                   # get grayscale image
imgBlurredDigit = cv2.GaussianBlur(imgGrayDigit, (5, 5), 0)                         # blur

# filter image from grayscale to black and white
imgThreshDigit = cv2.adaptiveThreshold(imgBlurredDigit,                             # input image
                                        # make pixels that pass the threshold full white
                                        255,
                                        # use gaussian rather than mean, seems to give better results
                                        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        # invert so foreground will be white, background will be black
                                        cv2.THRESH_BINARY_INV,
                                        # size of a pixel neighborhood used to calculate threshold value
                                        11,
                                        # constant subtracted from the mean or weighted mean
                                        2)

Find separate characters from digit image

In [ ]:
npaContoursDigit, _ = cv2.findContours(imgThreshDigit.copy(),       # input image, make sure to use a copy since the function will modify this image in the course of finding contours
                                       cv2.RETR_EXTERNAL,           # retrieve the outermost contours only
                                       cv2.CHAIN_APPROX_SIMPLE)     # compress horizontal, vertical, and diagonal segments and leave only their end points


Declare empty arrays

In [ ]:
AlphaContours = []
DigitContours = []

Create objects for alphabet contours

In [ ]:
for npaContour in npaContoursAlpha:                                 # for each contour
    # instantiate a contour with data object
    contourWithData = ContourWithData()
    # assign contour to contour with data
    contourWithData.npaContour = npaContour
    # get the bounding rect
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)
    # get bounding rect info
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()
    # calculate the contour area
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
    # add contour with data object to list of all contours with data
    if contourWithData.checkIfContourIsValid():
        AlphaContours.append(contourWithData)
    # end if
# end for

# sort contours from left to right
AlphaContours.sort(key = lambda x: x.intRectX)

Create objects for digit contours

In [ ]:
for npaContour in npaContoursDigit:                                 # for each contour
    # instantiate a contour with data object
    contourWithData = ContourWithData()
    # assign contour to contour with data
    contourWithData.npaContour = npaContour
    # get the bounding rect
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)
    # get bounding rect info
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()
    # calculate the contour area
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
    # add contour with data object to list of all contours with data
    if contourWithData.checkIfContourIsValid():
        DigitContours.append(contourWithData)
    # end if
# end for

# sort contours from left to right
DigitContours.sort(key=lambda x: x.intRectX)

Create empty arrays

In [ ]:
npaFlattenedImages = np.empty((0, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))

# declare empty classifications list, this will be our list of how we are classifying our chars from user input, we will write to file at the end
intClassifications = []

Label alphabet images

In [ ]:
char = 65
for i in range(len(AlphaContours)):                         # for each contour
    npaContour = AlphaContours[i]
    #increment character if new column
    if npaContour.intRectX - AlphaContours[i-1].intRectX >= AlphaContours[i-1].intRectWidth:
        char += 1
    # crop char out of threshold image
    imgROI = imgThreshAlpha[npaContour.intRectY: npaContour.intRectY + npaContour.intRectHeight,
                            npaContour.intRectX: npaContour.intRectX + npaContour.intRectWidth]
    # resize image, this will be more consistent for recognition and storage
    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
    # show training numbers image
    intClassifications.append(char)
    # flatten image to 1d numpy array so we can write to file later
    npaFlattenedImage = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
    # add current flattened impage numpy array to list of flattened image numpy arrays
    npaFlattenedImages = np.append(npaFlattenedImages, npaFlattenedImage, 0)
    # get bounding rectangle
    cv2.rectangle(imgTrainingAlpha, (npaContour.intRectX, npaContour.intRectY),
                    (npaContour.intRectX + npaContour.intRectWidth, npaContour.intRectY + npaContour.intRectHeight),
                    (0, 255, 0), 2)
    cv2.imshow("Training Alphabets", imgTrainingAlpha)
    print(chr(char), npaContour.intRectWidth)
    cv2.waitKey(500)
# end for
cv2.destroyAllWindows()
cv2.waitKey(5)

Label digit images

In [ ]:
char = 48
for i in range(len(DigitContours)):                      # for each contour
    npaContour = DigitContours[i]
    #increment character if new column
    if npaContour.intRectX - DigitContours[i-1].intRectX >= DigitContours[i-1].intRectWidth:
        char += 1
    # crop char out of threshold image
    imgROI = imgThreshDigit[npaContour.intRectY: npaContour.intRectY + npaContour.intRectHeight,
                            npaContour.intRectX: npaContour.intRectX + npaContour.intRectWidth]
    # resize image, this will be more consistent for recognition and storage
    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))
    # show training numbers image
    intClassifications.append(char)
    # flatten image to 1d numpy array so we can write to file later
    npaFlattenedImage = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))
    # add current flattened impage numpy array to list of flattened image numpy arrays
    npaFlattenedImages = np.append(npaFlattenedImages, npaFlattenedImage, 0)
    # get bounding rectangle
    cv2.rectangle(imgTrainingDigit, (npaContour.intRectX, npaContour.intRectY),
                  (npaContour.intRectX + npaContour.intRectWidth,
                   npaContour.intRectY + npaContour.intRectHeight),
                  (0, 255, 0), 2)
    cv2.imshow("Training Digits", imgTrainingDigit)
    print(chr(char), npaContour.intRectWidth)
# end for
cv2.destroyAllWindows()
cv2.waitKey(5)

Save training data

In [ ]:

fltClassifications = np.array(intClassifications, np.float32)

# flatten numpy array of floats to 1d so we can write to file later
npaClassifications = fltClassifications.reshape((fltClassifications.size, 1))

print("\n\ntraining complete !!\n")

# write flattened images to file
np.savetxt("classifications.txt", npaClassifications)
np.savetxt("flattened_images.txt", npaFlattenedImages)

# remove windows from memory
cv2.destroyAllWindows()

### TEST MODEL

Open classifications and flattened_images

In [4]:
# read in training classifications
try:
    npaClassifications = np.loadtxt("classifications.txt", np.float32)
except:
    print("error, unable to open classifications.txt, exiting program\n")
    exit(0)
# end try

# read in training images
try:
    npaFlattenedImages = np.loadtxt("flattened_images.txt", np.float32)
except:
    print("error, unable to open flattened_images.txt, exiting program\n")
    exit(0)
# end try

Reshape numpy array to 1d, necessary to pass to call to train

In [5]:
npaClassifications = npaClassifications.reshape((npaClassifications.size, 1))

Create KNN object

In [6]:
kNearest = cv2.ml.KNearest_create()         # instantiate KNN object
kNearest.train(npaFlattenedImages, cv2.ml.ROW_SAMPLE, npaClassifications)

True

Open testing image

In [16]:
img = cv2.imread(TEST_IMAGE)                # read in testing image
if img is None:                             # if image was not read successfully
    print("error: image not read from file \n\n")
    exit(0)
# end if

Edit image

In [8]:
# get grayscale image
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# blur image to reduce noise
imgBlurred = cv2.GaussianBlur(imgGray, (5, 5), 0)                    
# filter image from grayscale to black and white
imgThresh = cv2.adaptiveThreshold(imgBlurred,                       # input image
                                  255,                              # make pixels that pass the threshold full white
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,   # use gaussian rather than mean, seems to give better results
                                  cv2.THRESH_BINARY_INV,            # invert so foreground will be white, background will be black
                                  11,                               # size of a pixel neighborhood used to calculate threshold value
                                  2)                                # constant subtracted from the mean or weighted mean

Get separate contours from testing image

In [9]:
npaContours, npaHierarchy = cv2.findContours(# input image copy since the function will modify this image in the course of finding contours
                                             imgThresh.copy(),          
                                             # retrieve the outermost contours only
                                             cv2.RETR_EXTERNAL,         
                                             # compress horizontal, vertical, and diagonal segments and leave only their end points
                                             cv2.CHAIN_APPROX_SIMPLE)   

Create empty array

In [10]:
allContoursWithData = []                # store all valid contours in testing image

Create objects for each contour

In [11]:
for npaContour in npaContours:          # for each contour
    # instantiate a contour with data object
    contourWithData = ContourWithData()
    # assign contour to contour with data
    contourWithData.npaContour = npaContour
    # get the bounding rect
    contourWithData.boundingRect = cv2.boundingRect(contourWithData.npaContour)     
    # get bounding rect info
    contourWithData.calculateRectTopLeftPointAndWidthAndHeight()                    
    # calculate the contour area
    contourWithData.fltArea = cv2.contourArea(contourWithData.npaContour)
    # add contour with data object to list of all contours with data
    if contourWithData.checkIfContourIsValid():
        allContoursWithData.append(contourWithData)
# end for

Sort contours from left to right

In [12]:
# find maximum height of all contours
maxHeight = max(allContoursWithData, key=lambda x: x.intRectHeight).intRectHeight
print(maxHeight)
# sort contours from left to right
allContoursWithData.sort(key=lambda x: [np.ceil(x.intRectY / maxHeight), x.intRectX])

43


Declare final string, this will have the final number sequence by the end of the program

In [19]:
strFinalString = ""

In [20]:
for i in range(len(allContoursWithData)):            # for each contour
    # draw rectangle on original testing image
    contourWithData = allContoursWithData[i]
    cv2.rectangle(img,                                        
                  # upper left corner
                  (contourWithData.intRectX, contourWithData.intRectY),
                  (contourWithData.intRectX + contourWithData.intRectWidth,
                   contourWithData.intRectY + contourWithData.intRectHeight),       # lower right corner
                  (0, 255, 0),                                                      # green
                  2)                                                                # thickness

    # crop char out of threshold image
    imgROI = imgThresh[contourWithData.intRectY: contourWithData.intRectY + contourWithData.intRectHeight,     
                       contourWithData.intRectX: contourWithData.intRectX + contourWithData.intRectWidth]

    # resize image, this will be more consistent for recognition and storage
    imgROIResized = cv2.resize(imgROI, (RESIZED_IMAGE_WIDTH, RESIZED_IMAGE_HEIGHT))

    # flatten image into 1d numpy array
    npaROIResized = imgROIResized.reshape((1, RESIZED_IMAGE_WIDTH * RESIZED_IMAGE_HEIGHT))

    # convert from 1d numpy array of ints to 1d numpy array of floats
    npaROIResized = np.float32(npaROIResized)

    # call KNN function find_nearest
    retval, npaResults, neigh_resp, dists = kNearest.findNearest(npaROIResized, k=1)
    
    # get character from results
    strCurrentChar = str(chr(int(npaResults[0][0])))

    cv2.imshow("Test Image", img)
    print(strCurrentChar, contourWithData.intRectX, contourWithData.intRectY)

    if cv2.waitKey(500) == 27:
        cv2.destroyAllWindows()                         # remove windows from memory
        cv2.waitKey(10)
        break
    
    # check new line
    if contourWithData.intRectY  > allContoursWithData[i - 1].intRectY + contourWithData.intRectHeight:
        strFinalString += '\n'
    # end if
    
    # append current char to full string
    strFinalString = strFinalString + strCurrentChar
# end for

print("\n" + strFinalString + "\n")             # show final string
cv2.destroyAllWindows()                         # remove windows from memory
cv2.waitKey(10)

F 43 51
I 95 59
N 156 56
E 216 56
A 271 59
P 333 59
P 385 58
L 451 61
F 513 54
1 628 61
9 686 58
3 728 58
6 790 55
O 853 56
4 903 52
2 964 57
Y 1079 56
K 1139 60
C 1186 55
D 1247 52
R 43 115
A 104 118
N 151 121
D 209 122
O 274 128
M 325 124
W 386 121
0 447 124
R 503 121
D 558 121
S 621 121
8 681 121
9 740 125
2 790 124
0 856 125
4 914 122
1 971 126
7 1021 120
9 1087 118
3 1144 121
X 1197 123
Y 1261 118

FINEAPPLF1936O42YKCD
RANDOMW0RDS892041793XY



-1

: 